<a href="https://colab.research.google.com/github/Gatiwi93/Pronosticos/blob/main/Capitulo_3/Modelo_Regresi%C3%B3n_Lineal_Multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dependencias

In [1]:
# Importar librerias
from google.colab import drive
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

##Recopilación de datos

##Conjunto de datos: [Precio de las acciones de Meta durante los últimos 10 años](https://www.kaggle.com/datasets/aspillai/meta-stock-price-technical-indicators-10-years?select=meta_2014_2023.csv).

In [2]:
# Monta Google Drive en el entorno de Colab
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Ruta al archivo en tu Google Drive
ruta_archivo = '/content/drive/MyDrive/Recursos-Pronosticos/meta_2014_2023.csv'

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv(ruta_archivo)

# Mostrar las primeras filas del DataFrame
df


,date,open,high,low,close,volume,rsi_7,rsi_14,cci_7,cci_14,sma_50,ema_50,sma_100,ema_100,macd,bollinger,TrueRange,atr_7,atr_14,next_day_close
0,2014-01-02,54.830002,55.220001,54.189999,54.709999,43195500,51.917475,58.077822,-64.312117,-13.517101,50.2818,50.740946,47.665400,46.914555,1.828901,53.245000,1.030002,1.652052,1.710739,54.560001
1,2014-01-03,55.020000,55.650002,54.529999,54.560001,38246200,50.604988,57.387622,-40.054734,-17.361246,50.3194,50.890719,47.828800,47.066897,1.687987,53.542000,1.120003,1.576045,1.668543,57.200001
2,2014-01-06,54.419998,57.259998,54.049999,57.200001,68852600,67.483917,65.221525,43.907749,42.364730,50.4254,51.138151,48.030600,47.268784,1.768947,53.985000,3.209999,1.809467,1.778647,57.919998
3,2014-01-07,57.700001,58.549999,57.220001,57.919998,77207400,70.672584,67.003189,150.620136,117.886980,50.5348,51.404115,48.243300,47.480966,1.869653,54.484000,1.349998,1.743829,1.748030,58.230000
4,2014-01-08,57.599998,58.410000,57.230000,58.230000,56682400,72.049420,67.768804,107.795942,101.415192,50.6604,51.671806,48.460000,47.695073,1.951977,54.953500,1.180000,1.663282,1.707456,57.220001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,2023-12-22,355.579987,357.200012,351.220001,353.390015,11764200,75.599633,67.333880,76.743391,118.091091,325.3110,327.435849,313.733701,312.872285,6.988186,334.351501,5.980011,8.002103,8.111983,354.829987
2512,2023-12-26,354.989990,356.980011,353.450012,354.829987,9898600,77.018164,68.186993,77.588047,103.323614,326.1138,328.510129,314.150100,313.703131,7.482646,335.358000,3.589996,7.371802,7.788984,357.829987
2513,2023-12-27,356.070007,359.000000,355.309998,357.829987,13207900,79.863540,69.947896,105.699338,100.793824,326.8474,329.659928,314.621100,314.576930,8.024088,336.300000,4.170013,6.914403,7.530486,358.320007
2514,2023-12-28,359.700012,361.899994,357.809998,358.320007,11798800,80.327681,70.237681,130.191423,99.110535,327.5338,330.783852,315.038700,315.443129,8.395942,337.606000,4.089996,6.510917,7.284737,353.959991


##Ingeniería de Características

## Características (Variables Independientes):

* RSI_7: 7-day Relative Strength Index
* RSI_14: 14-day Relative Strength Index
* CCI_7: 7-day Commodity Channel Index
* CCI_14: 14-day Commodity Channel Index
* SMA_50: 50-day Simple Moving Average
* EMA_50: 50-day Exponential Moving Average
* SMA_100: 100-day Simple Moving Average
* EMA_100: 100-day Exponential Moving Average
* MACD: Moving Average Convergence Divergence
* Bollinger: Bollinger Bands
* TrueRange: True Range (volatilidad)
* ATR_7: 7-day Average True Range
* ATR_14: 14-day Average True Range



---



## Target (Variable Dependiente):

* Next_Day_Close: Precio de cierre del siguiente día




In [4]:
# Definir las variables independientes (características)
X = df[['rsi_7', 'rsi_14', 'cci_7', 'cci_14', 'sma_50', 'ema_50', 'sma_100', 'ema_100', 'macd', 'bollinger', 'TrueRange', 'atr_7', 'atr_14']]

# Definir la variable dependiente (objetivo)
y = df['next_day_close']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Regresión Lineal Multiple


In [5]:
# Inicializar y ajustar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir los valores de cierre para el próximo día en el conjunto de prueba
y_pred = model.predict(X_test)

# Coeficientes
print("Coeficientes: ", model.coef_)

# Intersección y
print("Intersección y: ", model.intercept_)

Coeficientes:  [ 0.11958744 -0.13791302  0.02094389  0.0061973   0.44347758  0.53847193
 -0.03647574 -0.09776455  2.708314    0.14321133 -0.06086559 -1.47971231
  1.74589992]
Intersección y:  1.23818996270424


In [6]:
# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

# Calcular el coeficiente de determinación (R cuadrado)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R cuadrado):", r2)


Error cuadrático medio: 28.381382133501923
Coeficiente de determinación (R cuadrado): 0.9955598762236557


## next_day_close	vs predicciones

In [7]:
# Crear un DataFrame con todas las características y el target
df_resultados = df.copy()  # Copiar el DataFrame original para mantener todas las características
df_resultados['predicciones'] = model.predict(X)  # Agregar las predicciones del modelo al DataFrame

# Mostrar el DataFrame con características, target y predicciones
df_resultados


,date,open,high,low,close,volume,rsi_7,rsi_14,cci_7,cci_14,...,ema_50,sma_100,ema_100,macd,bollinger,TrueRange,atr_7,atr_14,next_day_close,predicciones
0,2014-01-02,54.830002,55.220001,54.189999,54.709999,43195500,51.917475,58.077822,-64.312117,-13.517101,...,50.740946,47.665400,46.914555,1.828901,53.245000,1.030002,1.652052,1.710739,54.560001,54.360731
1,2014-01-03,55.020000,55.650002,54.529999,54.560001,38246200,50.604988,57.387622,-40.054734,-17.361246,...,50.890719,47.828800,47.066897,1.687987,53.542000,1.120003,1.576045,1.668543,57.200001,54.553868
2,2014-01-06,54.419998,57.259998,54.049999,57.200001,68852600,67.483917,65.221525,43.907749,42.364730,...,51.138151,48.030600,47.268784,1.768947,53.985000,3.209999,1.809467,1.778647,57.919998,57.776096
3,2014-01-07,57.700001,58.549999,57.220001,57.919998,77207400,70.672584,67.003189,150.620136,117.886980,...,51.404115,48.243300,47.480966,1.869653,54.484000,1.349998,1.743829,1.748030,58.230000,61.279028
4,2014-01-08,57.599998,58.410000,57.230000,58.230000,56682400,72.049420,67.768804,107.795942,101.415192,...,51.671806,48.460000,47.695073,1.951977,54.953500,1.180000,1.663282,1.707456,57.220001,60.859008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,2023-12-22,355.579987,357.200012,351.220001,353.390015,11764200,75.599633,67.333880,76.743391,118.091091,...,327.435849,313.733701,312.872285,6.988186,334.351501,5.980011,8.002103,8.111983,354.829987,350.650600
2512,2023-12-26,354.989990,356.980011,353.450012,354.829987,9898600,77.018164,68.186993,77.588047,103.323614,...,328.510129,314.150100,313.703131,7.482646,335.358000,3.589996,7.371802,7.788984,357.829987,353.464339
2513,2023-12-27,356.070007,359.000000,355.309998,357.829987,13207900,79.863540,69.947896,105.699338,100.793824,...,329.659928,314.621100,314.576930,8.024088,336.300000,4.170013,6.914403,7.530486,358.320007,356.768205
2514,2023-12-28,359.700012,361.899994,357.809998,358.320007,11798800,80.327681,70.237681,130.191423,99.110535,...,330.783852,315.038700,315.443129,8.395942,337.606000,4.089996,6.510917,7.284737,353.959991,359.462954


In [8]:
df_resultados.to_csv('/content/drive/MyDrive/Recursos-Pronosticos/Meta_Stock_Price_Technical_Indicators.csv')